In [1]:
import pandas as pd, os, numpy as np, requests, json
from time import time
%matplotlib inline

In [2]:
# create a new dataframe to contain the union of the combined spreadsheets, add year and full_address columns
entities = pd.DataFrame()
entities['year'] = np.nan
entities['full_address'] = ''

In [3]:
# specify necessary fields with a data type of string
data_types = {'ZIP':str, 'ID':str, 'COUNTRY':str}

In [4]:
# for each excel file in the data directory, load it and add its contents to the dataframe
for filename in os.listdir('data/'):
    if filename.endswith('.xls') or filename.endswith('.xlsx'):
        print filename,
        df_temp = pd.read_excel('data/' + filename, encoding='utf-8', converters=data_types)
        df_temp['year'] = filename[0:4]
        entities = pd.concat([entities, df_temp])

2005.xlsx 2006.xlsx 2007.xlsx 2008.xls 2009.xlsx 2010.xlsx 2011.xlsx 2012.xlsx 2013.xlsx 2014.xlsx 2015.xlsx


In [5]:
# rename columns to all lower case
entities_columns = ['address', 'city', 'country', 'id', 'last_update', 'name', 
                    'state', 'type', 'updated', 'x', 'y', 'zip', 'full_address', 'year']
entities.columns = entities_columns

In [6]:
# what state/province entities are represented in this data set?
entities['state'].unique()

array([u'MN', u'CA', u'CO', u'KS', u'MO', u'FL', u'IA', u'LA', u'NY',
       u'WI', u'SC', u'OR', u'AK', u'IN', u'VT', u'ME', u'WA', u'WY',
       u'IL', u'MA', u'NC', u'PA', u'MI', u'NM', u'GA', u'MD', u'VA',
       u'AZ', u'TN', u'MT', u'OH', u'NV', u'NJ', u'OK', u'ID', u'TX',
       u'WV', u'KY', u'UT', u'AR', u'HI', u'CT', u'DC', u'MS', u'RI',
       u'NE', u'SD', u'DE', u'NH', u'ND', u'AL', u'QC', u'ON', u'BC',
       u'AB', u'NB', u'SK', u'NL', u'PEI', u'NS', u'MB', u'YT', u'PE'], dtype=object)

In [7]:
# only retain U.S. rows: drop the rows with a canadian province in the state column
canadian_provinces = ['YT', 'PE', 'PEI', 'MB', 'NL', 'NB', 'NS', 'AB', 'BC', 'ON', 'SK', 'QC']
entities = entities[~entities['state'].isin(canadian_provinces)]

In [8]:
# how many records per year (includes duplicates at a location)
entities['year'].value_counts()

2015    4142
2014    3332
2013    2770
2012    2333
2011    2106
2010    1942
2009    1836
2008    1726
2007    1583
2006    1577
2005    1567
dtype: int64

In [9]:
# remove all rows that contain 'airport' or 'concourse' in their name or address column
entities = entities[~entities['name'].str.lower().str.contains('airport')]
entities = entities[~entities['address'].str.lower().str.contains('concourse').fillna(False)]

In [10]:
# extract street address without unit/etc, as cleanly as possible
regex_pattern = '#|,|\(|ste|suite|unit|warehouse|building|bldg'
entities['address1'] = entities['address'].str.lower().str.split(regex_pattern, return_type='frame')[0]

# create a full_address field to identify locations by street address
entities['full_address'] = entities['address1'].str.strip() + ' ' + entities['city'].str.strip() +  ' ' + entities['state'].str.strip()
entities['full_address'] = entities['full_address'].str.replace('.',  '').str.lower().str.strip()

# all done - drop the address1 column
entities.drop('address1', axis=1, inplace=True)

In [11]:
# which addresses the appear most often in the data set?
entities['full_address'].value_counts().head()

21001 n tatum blvd phoenix az            19
3702 se hawthorne blvd portland or       17
131 excelsior ave saratoga springs ny    17
925 s 3rd st la crosse wi                16
10426 e jomax rd scottsdale az           15
dtype: int64

In [12]:
# what entities are operating at the most common address?
most_common_address = entities['full_address'].value_counts().index[0]
entities[entities['full_address']==most_common_address].tail()

,address,city,country,id,last_update,name,state,type,updated,x,y,zip,full_address,year
1187,21001 N. Tatum Blvd.,Phoenix,NaN,1500,NaT,BJ's Brewhouse,AZ,BrewHouse,2011-10-17,-111.9774,33.6774,85050,21001 n tatum blvd phoenix az,2013
1772,21001 N. Tatum Blvd.,Phoenix,NaN,2273,NaT,Rock Bottom Restaurant & Brewery,AZ,BrewPub,2011-11-01,-111.9774,33.6774,85050,21001 n tatum blvd phoenix az,2013
384,21001 N. Tatum Blvd.,Phoenix,U.S.,1500,2011-10-15,BJ's Brewhouse,AZ,BrewHouse,NaT,-111.9774,33.6774,85050,21001 n tatum blvd phoenix az,2014
2847,21001 N. Tatum Blvd.,Phoenix,U.S.,2273,2013-05-15,Rock Bottom Restaurant & Brewery,AZ,BrewPub,NaT,-111.9774,33.6774,85050,21001 n tatum blvd phoenix az,2014
494,21001 N. Tatum Blvd.,Phoenix,U.S.,1500,2014-10-14,BJ’s Restaurant & Brewhouse,AZ,BrewHouse,NaT,-111.9774,33.6774,85050,21001 n tatum blvd phoenix az,2015


In [13]:
# create a new dataframe to capture locations, based on address, agnostic to entity operating at it
locations_columns = ['address', 'city', 'country', 'id', 'last_update', 
                     'name', 'state', 'type', 'updated', 'x', 'y', 'zip', 
                     'year', 'first_appearance', 'final_appearance', 'full_address']
locations = pd.DataFrame(columns = locations_columns)

In [14]:
start_time = time()
for _, row in entities.iterrows():
    
    # check if this address already appears in the locations
    matching_location = locations[locations['full_address'] == row['full_address']]
    
    # first check if this address appears multiple times - it shouldn't
    if len(matching_location) > 1:
        print 'problem: more than one row with this address'
      
    if len(matching_location) == 1:
        # address is already in there so check if years need updating
        loc_row = matching_location.index[0]
        
        # if the year is earlier in original data set then locations first year, update locations
        if int(row['year']) < int(locations.loc[loc_row, 'first_appearance']):
            locations.loc[loc_row, 'first_appearance'] = row['year']
        
        # if the year is later in original data set then locations final year, update locations
        if int(row['year']) > int(locations.loc[loc_row, 'final_appearance']):
            locations.loc[loc_row, 'final_appearance'] = row['year']
            
    else:
        # this address isn't in locations data set yet, so add it with this year
        loc_label = len(locations)
        locations.loc[loc_label, entities_columns] = row[entities_columns]
        locations.loc[loc_label, 'first_appearance'] = row['year']
        locations.loc[loc_label, 'final_appearance'] = row['year']
        
print 'process took %s seconds' % round(time() - start_time, 2)

process took 52.74 seconds


In [15]:
# sort data set by zip code
locations_cleaned = locations.sort(columns='zip', ascending=False)
locations_cleaned = locations_cleaned.reset_index()

In [16]:
# function to handle geocoding logic
def geocode(query):
    
    latlong = { }
    url = 'https://maps.googleapis.com/maps/api/geocode/json?sensor=false&address={0}&key={1}'
    apikey = '' #api key here
    request = url.format(query, apikey)
    data = json.loads(requests.get(request).text)
    if len(data['results']) > 0:
        latlong = {'lat':data['results'][0]['geometry']['location']['lat'] ,'lon':data['results'][0]['geometry']['location']['lng']}

    return latlong

In [17]:
# reset the full_address field to a geocodable string
locations_cleaned['full_address'] = locations_cleaned['address'].str.strip() + ', ' + locations_cleaned['city'].str.strip() +  ', ' + locations_cleaned['state'].str.strip() +  ', ' + locations_cleaned['zip'].str.strip()

In [18]:
# geocode address to lat-long for rows where lat-long is missing
count = 0
for label, row in locations_cleaned.iterrows():
    if pd.isnull(row['x']) | pd.isnull(row['y']):
        count = count + 1
        if label % 10 == 0: print label,
        latlong = geocode(row['full_address'])
        if latlong != { }:
            locations_cleaned.loc[label, 'y'] = latlong['lat']
            locations_cleaned.loc[label, 'x'] = latlong['lon']
print '\n' + str(count) + ' rows geocoded.'

10 50 90 100 130 180 200 290 330 340 370 380 390 490 520 620 660 690 700 730 740 750 790 860 900 920 930 940 970 1030 1110 1130 1160 1210 1220 1240 1390 1410 1500 1550 1590 1620 1650 1660 1700 1710 1790 1800 1860 1900 1940 1980 1990 2030 2070 2080 2160 2180 2220 2270 2280 2380 2410 2490 2530 2580 2590 2610 2650 2660 2670 2730 2760 2770 2780 2790 2870 2950 3000 3080 3090 3110 3150 3160 3230 3260 3310 3370 3380 3390 3430 3440 3470 3540 3710 3720 3800 3810 3830 3990 4020 4030 4040 4080 4130 4140 4150 4160 4190 4280 4290 4310 4320 4380 4430 4470 4520 4530 4540 4560 4580 4630 4740 4760 4780 4820 4890 4920 5020 5030 5040 5050 5070 5090 5110 5130 5160 5170 5190 5210 5220 5260 5370 5390 5410 5440 5450 5490 5520 5580 5640 5650 5660 
1576 rows geocoded.


In [19]:
# remove unnecessary columns
locations_cleaned = locations_cleaned.drop(axis=1, labels=['country', 'last_update', 'updated', 'year', 'index', 'full_address'])

In [20]:
# how many rows lack x or y?
len(locations_cleaned[pd.isnull(locations_cleaned['x']) | pd.isnull(locations_cleaned['y'])])

5

In [21]:
# drop all rows that lack lat or long (after we geocoded addresses earlier)
locations_cleaned['x'] = locations_cleaned['x'].astype(np.float)
locations_cleaned['y'] = locations_cleaned['y'].astype(np.float)
locations_cleaned = locations_cleaned[~(pd.isnull(locations_cleaned['x']) | pd.isnull(locations_cleaned['y']))]
locations_cleaned = locations_cleaned[~((locations_cleaned['x'] == 0) | (locations_cleaned['y'] == 0))]

In [22]:
# which states have the most locations in the data set?
locations_cleaned['state'].value_counts().head()

CA    821
CO    354
OR    345
WA    321
MI    282
dtype: int64

In [23]:
# how many locations made their first or their final appearances in each year?
pd.DataFrame({'first_appearance' : locations_cleaned['first_appearance'].value_counts(), 
              'final_appearance' : locations_cleaned['final_appearance'].value_counts()})

,final_appearance,first_appearance
2005,804,1557
2006,150,797
2007,80,178
2008,147,219
2009,76,233
2010,113,175
2011,106,274
2012,75,334
2013,120,504
2014,186,656


In [24]:
# show a slice of the data set
locations_cleaned.iloc[2100:2104]

,address,city,id,name,state,type,x,y,zip,first_appearance,final_appearance
2105,1427 W Elizabeth St,Fort Collins,NaN,C.B. & Potts Restaurants-Big Horn Breweries,CO,BrewPub,-105.102499,40.573884,80521,2005,2010
2106,133 Remington St.,Fort Collins,2606,Equinox Brewing Co.,CO,MicroBrewery,-105.075550,40.586520,80521,2011,2015
2107,470 Prospect Village Dr.,Estes Park,NaN,Estes Park Brewery,CO,BrewPub,-105.526121,40.371419,80517,2005,2015
2108,285 Cheesman St.,Erie,4185,Industrial Revolution Brewing Co.,CO,MicroBrewery,-105.048560,40.051518,80516,2014,2015


In [25]:
# save the data set to csv
locations_cleaned.to_csv('locations.csv', encoding='utf-8', index=False)

In [26]:
# add census fips code to each row - create new column to contain it
locations_cleaned['block_fips'] = None

In [27]:
# for each row in the dataframe
for label, row in locations_cleaned.iterrows():
    
    # if this row does not already have fips data
    if pd.isnull(row['block_fips']):
        
        # create a parameterized url and insert the latitude and longitude values as the parameters
        url = 'http://data.fcc.gov/api/block/find?format=json&latitude={0}&longitude={1}&showall=true'
        url = url.format(str(row['y']), str(row['x']))

        # fetch and load the JSON response from the API
        response = requests.get(url)
        data = json.loads(response.text)

        # save the data to this row in the dataframe
        locations_cleaned.loc[label, 'block_fips'] = data['Block']['FIPS']
    
    if label % 50 == 0: print label,

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900


In [28]:
# show the tail of the data set
locations_cleaned.tail()

,address,city,id,name,state,type,x,y,zip,first_appearance,final_appearance,block_fips
5913,3020 Water St.,Bay City,1581,Tri-City Brewing Co.,MI,MicroBrewery,-83.865100,43.608800,NaN,2007,2011,260172865001026
5914,2500 S. Harbor City Blvd.,Melbourne,NaN,Indian River Brewing Co.,FL,MicroBrewery,100.773042,18.775632,NaN,2008,2014,None
5915,NaN,Big Bend,NaN,Longnecks Brewpub & Restaurant,WI,NaN,100.773042,18.775632,NaN,2008,2008,None
5916,NaN,San Jose,2879,Shizmo Brewing Co. (by appt only),CA,MicroBrewery,-121.894955,37.339386,NaN,2011,2011,060855008001000
5917,NaN,Santa Cruz,3516,Lucky Hand Brewing,CA,MicroBrewery,100.773042,18.775632,NaN,2013,2013,None


In [29]:
len(locations_cleaned[pd.isnull(locations_cleaned['block_fips'])])

17

In [40]:
# fips code is 2 digit state, 3 digit county, 6 digit tract, 4 digit block (the first digit of which is the block group)
fips = locations_cleaned.loc[0, 'block_fips']
print 'fips', fips
print 'state', fips[0:2]
print 'county', fips[2:5]
print 'tract', fips[5:11]
print 'block', fips[11:15]

fips 021300001004012
state 02
county 130
tract 000100
block 4012


In [32]:
# add the tract fips as a new column to the data set
locations_cleaned['tract_fips'] = locations_cleaned['block_fips'].str.slice(start=0, stop=11)

In [33]:
# save the data set to csv
locations_cleaned.to_csv('locations-fips.csv', encoding='utf-8', index=False)

In [34]:
data_types = {'zip':str, 'block_fips':str, 'tract_fips':str}
locations_cleaned = pd.read_csv('locations-fips.csv', encoding='utf-8', converters=data_types)

In [35]:
# create a new dataframe to de-dupe locations, based on name + block_fips
locations_final_columns = ['address', 'city', 'id', 'name', 'state', 'type', 'x', 'y', 'zip', 
                           'first_appearance', 'final_appearance', 'block_fips', 'tract_fips', 'block_name']
locations_final = pd.DataFrame(columns = locations_final_columns)

In [36]:
locations_cleaned['block_name'] = locations_cleaned['block_fips'] + '+++' + locations_cleaned['name']
len(locations_cleaned)

5903

In [37]:
start_time = time()
for _, row in locations_cleaned.iterrows():
    
    # check if this block_name already appears in the locations_final
    matching_location = locations_final[locations_final['block_name'] == row['block_name']]
    
    # first check if this block_name appears multiple times - it shouldn't
    if len(matching_location) > 1:
        print 'problem: more than one row with this block_name'
      
    if len(matching_location) == 1:
        # address is already in there so check if years need updating
        match_label = matching_location.index[0]
        
        # if the year is earlier in original data set then locations first year, update locations
        if int(row['first_appearance']) < int(locations_final.loc[match_label, 'first_appearance']):
            locations_final.loc[match_label, 'first_appearance'] = row['first_appearance']
        
        # if the year is later in original data set then locations final year, update locations
        if int(row['final_appearance']) > int(locations_final.loc[match_label, 'final_appearance']):
            locations_final.loc[match_label, 'final_appearance'] = row['final_appearance']
            
    else:
        # this address isn't in locations data set yet, so add it with this year
        loc_label = len(locations_final)
        locations_final.loc[loc_label, locations_final_columns] = row[locations_final_columns]
        locations_final.loc[loc_label, 'first_appearance'] = row['first_appearance']
        locations_final.loc[loc_label, 'final_appearance'] = row['final_appearance']
        
print 'process took %s seconds' % round(time() - start_time, 2)
len(locations_final)

process took 13.92 seconds


5458

In [38]:
# save the data set to csv
locations_final = locations_final.drop(axis=1, inplace=False, labels=['block_name'])
locations_final = locations_final.sort('block_fips')
locations_final.to_csv('locations-final.csv', encoding='utf-8', index=False)

In [39]:
# save a list of block_fips that appear more than once in the data set, to csv
dupe_block_fips = locations_final[locations_final.duplicated(subset=['block_fips'])]['block_fips']
dupe_block_fips.to_csv('dupe-block_fips.csv', encoding='utf-8', index=False)